## Agentes de IA generativa

### ¿Que vamos a ver?


Usando el framework agents SDK de openAI

0. ¿Por que usar un framework?
1. Creación de un agente simple con los MCP definidos anteriormente + human in the loop
2. Añadimos un segundo agente mediante handover para hacer busquedas web
    
    2.1 Memoria a corto plazo y manejo del contexto.
3. Partimos el agente en un sistema de agentes múltiples con un supervisor
    
    3.1 Handoffs vs Agent as a tool
4. Añadimos un planificador inicial y paralelismo.
5. Un vistazo a otros frameworks (langgraph, A2A, Semantic Kernel)

In [ ]:
%pip install openai-agents
%pip install --upgrade openai 

Creamos un "hola mundo" con el sdk.

In [ ]:
import os
import openai
import agents

os.environ["AZURE_OPENAI_API_KEY"]=" "
os.environ["AZURE_OPENAI_ENDPOINT"]=" "
os.environ["OPENAI_API_VERSION"]="2025-04-01-preview"
from agents import Agent, Runner
client = openai.AsyncAzureOpenAI(api_key=os.environ["AZURE_OPENAI_API_KEY"], api_version=os.environ["OPENAI_API_VERSION"], azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"])
azure_model = agents.OpenAIChatCompletionsModel(openai_client=client, model="gpt-5-mini")

agent = Agent(name="Assistant", instructions="You are a helpful assistant", model=azure_model)
question = input("What do you want to know?")
result = await Runner.run(agent, question)
print(result.final_output)

Añadamos los mcps 

Nota: Con esta configuración, la "ñ", da problemas. Hay que quitarla

Nota 2: Añadimos observabilidad desde el principio

In [ ]:
import logfire
logfire.configure(token=" ")
logfire.instrument_openai_agents() 
from agents.mcp import MCPServer, MCPServerStreamableHttp
async with  MCPServerStreamableHttp(
    name = "User Wallet",
    params = {
    "url" : "http://localhost:800/mcp/"
    }
) as wallet_mcp:
    agent = Agent(name="Assistant", instructions="You are a helpful assistant", model=azure_model, mcp_servers=[wallet_mcp])
    question = input("What do you want to know?")
    result = await Runner.run(agent, question)
    print(result.final_output)


Currently retrying 1 failed export(s)


Añadimos el otro server y organizamos un poco el código

In [ ]:
async def run_agents(mcp_servers, user_input):
    for server in mcp_servers:
        await server.connect()
    agent = Agent(name="Assistant", instructions="Eres un asistente financiero que puede hacer operaciones con el portafolio de un usuario", model=azure_model, mcp_servers=mcp_servers)
    result = await Runner.run(agent, user_input)
    return result.final_output

async def create_mcps():
    server_list = []
    async with MCPServerStreamableHttp(
        name="User Wallet",
        params={
            "url": "http://localhost:800/mcp/"
        }
    ) as wallet_mcp:
        wallet_mcp.connect()
        server_list.append(wallet_mcp)
        async with MCPServerStreamableHttp(
            name="Financial live data",
            params={
                "url": "http://localhost:8002/mcp/"
            }
        ) as financial_mcp:
            financial_mcp.connect()
            server_list.append(financial_mcp)
            return server_list

async def main():
    question = input("En que puedo ayudarte?")
    server_list = await create_mcps()
    res = await run_agents(server_list, question)

await main() #Nota, fuera de jupyter, se debe usar asyncio.run(main())

Vamos a añadir human in the loop y a meter un numero máximo de ejecuciones

In [ ]:
from agents.agent import StopAtTools
from agents import Agent, FunctionTool, RunContextWrapper, function_tool
async def run_agents(mcp_servers, user_input):
    
    @function_tool
    def human_in_the_loop(question: str):
        '''
        Esta tool te permite preguntar al usuario. No intentes preguntar al usuario sin usar esta tool
        '''
        return input(question)
    
    for server in mcp_servers:
        await server.connect()
    agent = Agent(name="Assistant", 
                  instructions='''Eres un asistente financiero que puede hacer operaciones con el portafolio de un usuario, 
                  tienes una tool human_in_the_loop que te permite preguntar al usuario. No intentes preguntar al usuario sin usar esa tool ''', 
                  model=azure_model, 
                  mcp_servers=mcp_servers,
                  tools=[human_in_the_loop],

                  )
    result = await Runner.run(agent, user_input)
    return result.final_output

async def create_mcps():
    server_list = []
    async with MCPServerStreamableHttp(
        name="User Wallet",
        params={
            "url": "http://localhost:800/mcp/"
        }
    ) as wallet_mcp:
        wallet_mcp.connect()
        server_list.append(wallet_mcp)
        async with MCPServerStreamableHttp(
            name="Financial live data",
            params={
                "url": "http://localhost:8002/mcp/"
            }
        ) as financial_mcp:
            financial_mcp.connect()
            server_list.append(financial_mcp)
            return server_list

async def main():
    question = input("En que puedo ayudarte?")
    server_list = await create_mcps()
    res = await run_agents(server_list, question)

await main() #Nota, fuera de jupyter, se debe usar asyncio.run(main())

Vamos a añadir un segundo agente que haga busquedas web. Implementaremos una estrategia de handoff para que la ejecución pase a este segundo agente. 
OpenAI ya tiene busquedas web, pero para hacerlo agnostico respecto al modelo, utilizaremos Tavily, que es un proveedor externo.

In [ ]:
tavily_api_key = "tvly-dev- "
tavily_mcp_url = f"https://mcp.tavily.com/mcp/?tavilyApiKey={tavily_api_key}"


In [ ]:
from agents.agent import StopAtTools
from agents import Agent, FunctionTool, RunContextWrapper, function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents.model_settings import ModelSettings


user_wallet_mcp = MCPServerStreamableHttp(
        name="User Wallet",
        params={
            "url": "http://localhost:800/mcp/"
        },
        cache_tools_list=True
    ) 
financial_mcp = MCPServerStreamableHttp(
    name="Financial live data",
    params={
        "url": "http://localhost:8002/mcp/"
    },
    cache_tools_list=True
) 
web_search_mcp = MCPServerStreamableHttp(
    name="Web search",
    params={
        "url": "https://mcp.tavily.com/mcp/?tavilyApiKey=tvly-dev- "
    },
    client_session_timeout_seconds=10.0,
    cache_tools_list=True
)

async def run_agents(user_input):
    @function_tool
    def human_in_the_loop(question: str):
        '''
        Esta tool te permite preguntar al usuario. No intentes preguntar al usuario sin usar esta tool
        '''
        return input(question)
 
    async with user_wallet_mcp, financial_mcp, web_search_mcp:

        wallet_agent = Agent(name="wallet_agent", 
                    instructions='''Eres un asistente financiero que puede hacer operaciones con el portafolio de un usuario, 
                    tienes una tool human_in_the_loop que te permite preguntar al usuario o comunicar tus conclusiones. No intentes comunicarte con el usuario sin usar esa tool ''', 
                    model=azure_model, 
                    tools=[human_in_the_loop],
                    mcp_servers=[user_wallet_mcp],


                    )
        research_agent = Agent(name="research_agent", 
                            instructions='''Eres un agente de investigación financiera que puede 
                            hacer busquedas web mediante la tool web_search_mcp.
                            Tienes una tool human_in_the_loop que te permite preguntar al usuario o coumunicar tus conclusiones. No intentes comunicarte con el usuario sin usar esa tool''',
                            model=azure_model,
                            mcp_servers=[web_search_mcp, financial_mcp],
                            tools=[human_in_the_loop],
                            )
        assistant_agent = Agent(name="Assistant", 
                    instructions=f'''{RECOMMENDED_PROMPT_PREFIX}
                    Eres un asistente financiero que puede resolver varias tareas apoyandose en otros agentes especializados. 
                    tienes una tool human_in_the_loop que te permite preguntar al usuario. No intentes preguntar al usuario sin usar esa tool.
                    Delega la conversación a un agente de investigación financiera si necesitas hacer una busqueda web o investigar datos de una accion en tiempo real.
                    Delega la conversación a un agente de gestion de wallet si necesitas hacer operaciones con el portafolio de un usuario.
                    Recuerda que tu conocimiento es limitado y no está actualizado, por lo que debes delegar las tareas a los agentes especializados.
                    ''', 
                    model=azure_model, 
                    tools=[human_in_the_loop],
                    handoffs=[research_agent, wallet_agent],
                    )
        

        result = await Runner.run(assistant_agent, user_input, max_turns=30)
        return result.final_output


async def main():
    # question = input("En que puedo ayudarte?")
    question = "Invierte todo mi portfolio en algo que los analistas online consideren interesante. No me preguntes mucho."
    res = await run_agents(question)

await main() #Nota, fuera de jupyter, se debe usar asyncio.run(main())

Con este diseño conocido como "Triaje", un agente inicial decide a quien delegar la tarea, el problema es que cuando el agente investigador termina su parte, no puede llamar al agente que maneja la wallet para que ejecute la orden.

Este diseño es un patrón interesante para tareas de soporte al cliente, en las que sea admisible que un cliente se vaya derivando a distintos especialistas IA y se comunique con ellos, pero no es tan interesante cuando queremos que se realice una tarea compleja de manera autónoma o semi autónoma.

Por otro lado ¿Crees que el agente investigador tiene su responsabilidad bien acotada o crees que se extralimita?

Para resolver esto podríamos:
- Darle a todos los agentes la capacidad de hacer handoff en cualquiera de ellos.
- Darle a todos los agentes la capacidad de usar la wallet
- Obligar a los agentes a volver al asistente al terminar con el resultado.

Implementemos esta última. En la librería que usamos la manera de hacerlo es usar "agents-as-tools"

In [ ]:
from agents.agent import StopAtTools
from agents import Agent, FunctionTool, RunContextWrapper, function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents.model_settings import ModelSettings


user_wallet_mcp = MCPServerStreamableHttp(
        name="User Wallet",
        params={
            "url": "http://localhost:800/mcp/"
        },
        cache_tools_list=True
    ) 
financial_mcp = MCPServerStreamableHttp(
    name="Financial live data",
    params={
        "url": "http://localhost:8002/mcp/"
    },
    cache_tools_list=True
) 
web_search_mcp = MCPServerStreamableHttp(
    name="Web search",
    params={
        "url": "https://mcp.tavily.com/mcp/?tavilyApiKey=tvly-dev- "
    },
    client_session_timeout_seconds=10.0,
    cache_tools_list=True
)

async def run_agents(user_input):
    
    @function_tool
    def human_in_the_loop(question: str):
        '''
        Esta tool te permite preguntar al usuario. No intentes preguntar al usuario sin usar esta tool
        '''
        return input(question)
    async with user_wallet_mcp, financial_mcp, web_search_mcp:

    
        wallet_agent = Agent(name="wallet_agent", 
                    instructions='''Eres un asistente financiero que puede hacer operaciones con el portafolio de un usuario. 
                    Si se te pide algo que no puedes hacer, explica al usuario que no puedes hacerlo y si puedes sugiere 
                    algo similar que si puedes hacer. Todas tus capacidades son las que te ofrece el mcp de user_wallet_mcp''',
                    model=azure_model,
                    mcp_servers=[user_wallet_mcp],


                    )
        research_agent = Agent(name="research_agent", 
                            instructions='''Eres un agente de investigación financiera que puede 
                            hacer busquedas web mediante la tool web_search_mcp. Tu objetivo es investigar, no hacer operaciones ni sacar conclusiones.
                            Tienes un máximo de 10 intentos para investigar.
                            Si ya los has usado, responde con lo que has investigado hasta el momento.
                            ''',
                            model=azure_model,
                            mcp_servers=[web_search_mcp, financial_mcp],
                            )
        assistant_agent = Agent(name="Assistant", 
                    instructions='''
                    Eres un asistente financiero que puede resolver varias tareas apoyandose en otros agentes especializados. 
                    tienes una tool human_in_the_loop que te permite preguntar al usuario. No intentes preguntar al usuario sin usar esa tool.
                    Delega la conversación a research_agent si necesitas hacer una busqueda web o investigar datos de una accion en tiempo real.
                    Delega la conversación a wallet_agent si necesitas hacer operaciones con el portafolio de un usuario.
                    Recuerda que tu conocimiento es limitado y no está actualizado, por lo que debes delegar las tareas a los agentes especializados.
                    Tu coordinas a los agentes, extraes conclusiones y eres el único capaz de comunicarte con el usuario.
                    ''', 
                    model=azure_model, 
                    tools=[human_in_the_loop, 
                        research_agent.as_tool(
                            tool_name="research_agent",
                            tool_description="Busca información en la web o investiga datos de una accion en tiempo real"
                        ), 
                        wallet_agent.as_tool(
                            tool_name="wallet_agent",
                            tool_description="Hace operaciones con el portafolio de un usuario"
                        )],
                    )
        

        result = await Runner.run(assistant_agent, user_input, max_turns=30)
        return result.final_output


async def main():
    # question = input("En que puedo ayudarte?")
    question = "Invierte todo mi portfolio en algo que los analistas online consideren interesante. No me preguntes mucho."
    res = await run_agents(question)

await main() #Nota, fuera de jupyter, se debe usar asyncio.run(main())

Tiene muy buena pinta, pero fijaos que los agentes agotan sus turnos con mucha facilidad (menos mal que los hemos limitado). Vamos a ver si podemos darle algunos turnos mas y poner algun fallback.

In [ ]:
from agents.agent import StopAtTools
from agents import Agent, FunctionTool, RunContextWrapper, function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents.model_settings import ModelSettings
from agents.exceptions import MaxTurnsExceeded, RunErrorDetails

user_wallet_mcp = MCPServerStreamableHttp(
        name="User Wallet",
        params={
            "url": "http://localhost:800/mcp/"
        },
        cache_tools_list=True
    ) 
financial_mcp = MCPServerStreamableHttp(
    name="Financial live data",
    params={
        "url": "http://localhost:8002/mcp/"
    },
    cache_tools_list=True
) 
web_search_mcp = MCPServerStreamableHttp(
    name="Web search",
    params={
        "url": "https://mcp.tavily.com/mcp/?tavilyApiKey=tvly-dev- "
    },
    client_session_timeout_seconds=10.0,
    cache_tools_list=True
)

async def run_agents(user_input):
    @function_tool
    def human_in_the_loop(question: str):
        '''
        Esta tool te permite preguntar al usuario. No intentes preguntar al usuario sin usar esta tool
        '''
        return input(question)
    



    
    async with user_wallet_mcp, financial_mcp, web_search_mcp:

        @function_tool
        async def research_agent(input: str) -> str:
            """Busca información en la web o investiga datos de una accion en tiempo real"""

            research_agent = Agent(name="research_agent", 
                instructions='''Eres un agente de investigación financiera que puede 
                hacer busquedas web mediante la tool web_search_mcp. Tu objetivo es investigar, no hacer operaciones ni sacar conclusiones.
                ''',
                model=azure_model,
                mcp_servers=[web_search_mcp, financial_mcp],
                )
            try:
                result = await Runner.run(
                    research_agent,
                    input=input,
                    max_turns=15,
                )
                res = result.final_output
            except MaxTurnsExceeded as e:
                res = str(e.run_data)
            return str(res)
    
        wallet_agent = Agent(name="wallet_agent", 
                    instructions='''Eres un asistente financiero que puede hacer operaciones con el portafolio de un usuario. 
                    Si se te pide algo que no puedes hacer, explica al usuario que no puedes hacerlo y si puedes sugiere 
                    algo similar que si puedes hacer. Todas tus capacidades son las que te ofrece el mcp de user_wallet_mcp''',
                    model=azure_model,
                    mcp_servers=[user_wallet_mcp],


                    )

        assistant_agent = Agent(name="Assistant", 
                    instructions='''
                    Eres un asistente financiero que puede resolver varias tareas apoyandose en otros agentes especializados. 
                    tienes una tool human_in_the_loop que te permite preguntar al usuario. No intentes preguntar al usuario sin usar esa tool.
                    Delega la conversación a research_agent si necesitas hacer una busqueda web o investigar datos de una accion en tiempo real.
                    Delega la conversación a wallet_agent si necesitas hacer operaciones con el portafolio de un usuario.
                    Recuerda que tu conocimiento es limitado y no está actualizado, por lo que debes delegar las tareas a los agentes especializados.
                    Tu coordinas a los agentes, extraes conclusiones y eres el único capaz de comunicarte con el usuario.
                    ''', 
                    model=azure_model, 
                    tools=[human_in_the_loop, research_agent,
                        wallet_agent.as_tool(
                            tool_name="wallet_agent",
                            tool_description="Hace operaciones con el portafolio de un usuario"
                        )],
                    )
        

        result = await Runner.run(assistant_agent, user_input, max_turns=30)
        return result.final_output


async def main():
    # question = input("En que puedo ayudarte?")
    question = "Invierte todo mi portfolio en algo que los analistas online consideren interesante. No me preguntes mucho."
    res = await run_agents(question)

await main() #Nota, fuera de jupyter, se debe usar asyncio.run(main())

En el sistema que estamos construyendo, la parte mas importante (y quizá la mas compleja) es la que hace la investigación. Vamos a tratar de hacer algo mas elaborado:
- Transformar el investigador en un "equipo" de investigadores.
- El que inicia la investigación es el "jefe" y va a comenzar con una planificación
- A partir de la planificación, intentaremos paralelizar lo que sea posible.

Por simplicidad, voy a dejar fuera la gestión de la wallet y voy a hacer el ejemplo acotado a esto, sería trivial añadir de nuevo el agente de la wallet si te apetece.

In [ ]:

from agents import Agent, FunctionTool, RunContextWrapper, function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents.model_settings import ModelSettings
from agents.exceptions import MaxTurnsExceeded, RunErrorDetails

modelo_listo = agents.OpenAIChatCompletionsModel(openai_client=client, model="gpt-5")

financial_mcp = MCPServerStreamableHttp(
    name="Financial live data",
    params={
        "url": "http://localhost:8002/mcp/"
    },
    cache_tools_list=True
) 
web_search_mcp = MCPServerStreamableHttp(
    name="Web search",
    params={
        "url": "https://mcp.tavily.com/mcp/?tavilyApiKey=tvly-dev- "
    },
    client_session_timeout_seconds=10.0,
    cache_tools_list=True
)

async def run_agents(user_input):
    @function_tool
    def human_in_the_loop(question: str):
        '''
        Esta tool te permite preguntar al usuario. No intentes preguntar al usuario sin usar esta tool
        '''
        return input(question)
    



    
    async with  financial_mcp, web_search_mcp:

        @function_tool
        async def research_agent(input: str) -> str:
            """Agente que busca información en la web o investiga datos de una accion en tiempo real"""

            research_agent = Agent(name="research_agent", 
                instructions='''Eres un agente de investigación financiera que puede 
                hacer busquedas web mediante la tool web_search_mcp. Tu objetivo es investigar, no hacer operaciones ni sacar conclusiones.
                Puedes probar diferentes palabras clave para encontrar la información que necesitas, pero no hagas demasiados experimentos.
                Si crees que no puedes completar la tarea, responde con lo que has investigado hasta el momento y lo que has hecho para buscarlo 
                y el que te mandó la tarea sabrá como solucionarlo.
                ''',
                model=azure_model,

                mcp_servers=[web_search_mcp, financial_mcp],
                )
            try:
                result = await Runner.run(
                    research_agent,
                    input=input,
                    max_turns=15,
                )
                res = result.final_output
            except MaxTurnsExceeded as e:
                res = str(e.run_data)
            return str(res)
    

        assistant_agent = Agent(name="Assistant", 
                    instructions='''
                    Eres el jefe de un equipo de investigadores financieros. El cliente te va a plantear una consulta.
                    Para resolverla, debes:
                    - Escribir un plan de investigación, en el que partas la investigacion en varias investigaciones parciales.
                    - Discrimina bien que partes son parelilzables y cuales son secuenciales.
                    - Usando human_in_the_loop, confirma el plan con el cliente. También puedes usarla para pedirle 
                    que te ayude a escribir el plan. Pero ten en cuenta que el cliente tiene un conocimiento muy basico de finanzas
                    - Cuando estés listo, delegas las partes en paralelo a los investigadores llamando a research_agent.
                    - A medida que los investigadores van encontrando información, vas recopilando las conclusiones.
                    - Puedes revisar el plan si algo no va bien, pero comunicaselo al cliente.

                    ''', 
                    model=modelo_listo, 
                    model_settings=ModelSettings(reasoning={"effort": "low"}),
                    tools=[human_in_the_loop, research_agent]
                    )
        

        result = await Runner.run(assistant_agent, user_input, max_turns=30)
        return result.final_output


async def main():
    # question = input("En que puedo ayudarte?")
    question = "Tengo 1000€, dime si es mejor invertir en bitcoin, oro, materias primas o alguna empresa de defensa"
    res = await run_agents(question)

await main() #Nota, fuera de jupyter, se debe usar asyncio.run(main())

17:46:43.928 OpenAI Agents trace: Agent workflow
17:46:43.936   Agent run: 'Assistant'
17:46:43.937     Chat completion with 'gpt-5' [LLM]
17:46:58.495     Function: human_in_the_loop
17:49:53.145     Chat completion with 'gpt-5' [LLM]
17:50:13.165     Function: human_in_the_loop
17:50:52.863     Chat completion with 'gpt-5' [LLM]
17:51:02.950     Function: research_agent
17:51:02.954     Function: research_agent
17:51:02.955     Function: research_agent
17:51:02.955     Function: research_agent
                 Function: research_agent
17:51:02.961       MCP: list tools from server Web search
                 Function: research_agent
17:51:02.965       MCP: list tools from server Web search
                 Function: research_agent
17:51:02.966       MCP: list tools from server Web search
                 Function: research_agent
17:51:02.966       MCP: list tools from server Web search
17:51:03.283       MCP: list tools from server Financial live data
                 Function: resea

Error invoking MCP tool tavily_search: Timed out while waiting for response to ClientRequest. Waited 10.0 seconds.


                 Function: research_agent
                   Agent run: 'research_agent'
17:51:25.126         Function: tavily_search
17:51:27.292         MCP: list tools from server Web search
17:51:27.292         MCP: list tools from server Financial live data
17:51:27.297         Chat completion with 'gpt-5-mini' [LLM]
                 Function: research_agent
                   Agent run: 'research_agent'
17:51:27.365         Function: tavily_extract
17:51:28.112         MCP: list tools from server Web search
17:51:28.115         MCP: list tools from server Financial live data
17:51:28.118         Chat completion with 'gpt-5-mini' [LLM]
                 Function: research_agent
                   Agent run: 'research_agent'
17:51:28.256         Function: tavily_search
                 Function: research_agent
                   Agent run: 'research_agent'
17:51:29.640         Function: tavily_search
                 Function: research_agent
                   Agent run: 'research_a

Error invoking MCP tool tavily_search: Timed out while waiting for response to ClientRequest. Waited 10.0 seconds.


                 Function: research_agent
                   Agent run: 'research_agent'
17:51:39.427         Function: tavily_search
                 Function: research_agent
                   Agent run: 'research_agent'
17:51:39.668         Function: tavily_search
                 Function: research_agent
                   Agent run: 'research_agent'
17:51:41.310         MCP: list tools from server Web search
17:51:41.316         MCP: list tools from server Financial live data
17:51:41.326         Chat completion with 'gpt-5-mini' [LLM]
                 Function: research_agent
                   Agent run: 'research_agent'
17:51:42.889         MCP: list tools from server Web search
17:51:42.894         MCP: list tools from server Financial live data
17:51:42.898         Chat completion with 'gpt-5-mini' [LLM]
                 Function: research_agent
                   Agent run: 'research_agent'
17:51:44.677         Function: tavily_search
                 Function: research_agen

Error invoking MCP tool tavily_search: Timed out while waiting for response to ClientRequest. Waited 10.0 seconds.


17:52:06.981         Function: tavily_search
17:52:08.634         MCP: list tools from server Web search
17:52:08.635         MCP: list tools from server Financial live data
17:52:08.640         Chat completion with 'gpt-5-mini' [LLM]
17:52:15.476         Function: tavily_search
17:52:17.658         MCP: list tools from server Web search
17:52:17.661         MCP: list tools from server Financial live data
17:52:17.667         Chat completion with 'gpt-5-mini' [LLM]
17:52:20.900         Function: tavily_search
17:52:23.963         MCP: list tools from server Web search
17:52:23.965         MCP: list tools from server Financial live data
17:52:23.971         Chat completion with 'gpt-5-mini' [LLM]
17:52:26.790         Function: tavily_search
17:52:29.017         MCP: list tools from server Web search
17:52:29.019         MCP: list tools from server Financial live data
17:52:29.029     Chat completion with 'gpt-5' [LLM]


In [13]:
%pip install "openai-agents[viz]"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\ginesrodrigo.sanchez\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
